## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib
# import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [11]:
import tensorflow as tf

from keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, Input
from tensorflow import feature_column
from tensorflow.keras.layers.experimental import preprocessing

In [3]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

### Loading data

In [4]:
raw_data = pd.read_csv('7342422066d511eb/dataset/train.csv')

#pandas profiling

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
profile = ProfileReport(raw_data, title='Pandas Profiling Report', explorative=True)
profile

#### dropping some col

In [5]:
raw_data = raw_data.drop(['Customer Id', 'Artist Reputation', 'Height', 'Width',
                           'Weight','Material', 'Transport', 'Remote Location', 'Scheduled Date', 'Delivery Date'
                         ,'Customer Location'],axis = 1) 

In [6]:
raw_data.rename(columns= {'Price Of Sculpture': 'sculpture_price',
                          'Base Shipping Price':'shipping_price',
                          'Artist Name':'artist_name',
                          'Express Shipment':'express_shipment',
                          'Installation Included':'installation',
                             'Customer Information': 'customer_info'
                         }, inplace=True)
raw_data.head(10)

,artist_name,sculpture_price,shipping_price,International,express_shipment,installation,Fragile,customer_info,Cost
0,Billy Jenkins,13.91,16.27,Yes,Yes,No,No,Working Class,-283.29
1,Jean Bryant,6.83,15.00,No,No,No,No,Working Class,-159.96
2,Laura Miller,4.96,21.18,No,No,No,Yes,Working Class,-154.29
3,Robert Chaires,5.81,16.31,No,No,No,No,Wealthy,-161.16
4,Rosalyn Krol,3.18,11.94,Yes,Yes,Yes,No,Working Class,-159.23
5,Tracy Francis,6.16,16.88,No,Yes,No,No,Wealthy,-1922.78
6,David Hawes,8.26,90.67,No,Yes,No,No,Working Class,-1536.66
7,David Osher,12.81,13.25,Yes,No,No,Yes,Wealthy,-422.42
8,Arnold Reel,3.98,19.76,Yes,No,No,Yes,Working Class,-160.10
9,James Comfort,245.63,49.25,No,No,No,No,Working Class,-834.27


In [7]:
artist = pd.get_dummies(raw_data.artist_name, prefix='artist_name')
express = pd.get_dummies(raw_data.express_shipment, prefix='express_shipment')

installation = pd.get_dummies(raw_data.installation, prefix='installation')
fragile = pd.get_dummies(raw_data.Fragile, prefix='Fragile')

customer_info = pd.get_dummies(raw_data.customer_info, prefix='customer_info')

In [8]:
X = pd.concat([artist, express, installation, fragile, customer_info, 
               raw_data['sculpture_price'], raw_data['shipping_price']] , axis=1)
Y = raw_data['Cost']

## splitting data into train, test and validation sets

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=0)

# normalization
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Model

In [15]:
input_layer = Input(shape=(X.shape[1],))
dense_layer_1 = Dense(100, activation='relu')(input_layer)
dense_layer_2 = Dense(50, activation='relu')(dense_layer_1)
dense_layer_3 = Dense(25, activation='linear')(dense_layer_2)
output = Dense(1)(dense_layer_3)

model = Model(inputs=input_layer, outputs=output)
model.compile(loss="mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])

In [13]:
# model = tf.keras.Sequential([
#   feature_layer,
#   layers.Dense(16, activation='relu'),
#   layers.Dense(16, activation='relu'),
#   layers.Dense(16, activation='relu'),
#   layers.Dropout(.1),
#   layers.Dense(1, activation='linear')
# ])

In [14]:
# model.compile(optimizer='adam',
#               loss=tf.keras.losses.mse,
#               metrics=['accuracy'])

In [16]:
history = model.fit(X_train, y_train, batch_size=2, epochs=100, verbose=1, validation_split=0.2)

Epoch 1/100
2210/2210 [==============================] - 8s 4ms/step - loss: 54990447538.9119 - mean_squared_error: 54990447538.9119 - val_loss: 34213675008.0000 - val_mean_squared_error: 34213675008.0000
Epoch 2/100
2210/2210 [==============================] - 9s 4ms/step - loss: 25581744413.5106 - mean_squared_error: 25581744413.5106 - val_loss: 33679259648.0000 - val_mean_squared_error: 33679259648.0000
Epoch 3/100
2210/2210 [==============================] - 9s 4ms/step - loss: 18250684548.1409 - mean_squared_error: 18250684548.1409 - val_loss: 38709252096.0000 - val_mean_squared_error: 38709252096.0000
Epoch 4/100
2210/2210 [==============================] - 8s 4ms/step - loss: 40274646263.9530 - mean_squared_error: 40274646263.9530 - val_loss: 38023188480.0000 - val_mean_squared_error: 38023188480.0000
Epoch 5/100
2210/2210 [==============================] - 8s 4ms/step - loss: 31634246387.5097 - mean_squared_error: 31634246387.5097 - val_loss: 57015373824.0000 - val_mean_squared

2210/2210 [==============================] - 15s 7ms/step - loss: 634717002.6386 - mean_squared_error: 634717002.6386 - val_loss: 4750442496000.0000 - val_mean_squared_error: 4750442496000.0000
Epoch 41/100
2210/2210 [==============================] - 14s 6ms/step - loss: 652345970.9688 - mean_squared_error: 652345970.9688 - val_loss: 4292116742144.0000 - val_mean_squared_error: 4292116742144.0000
Epoch 42/100
2210/2210 [==============================] - 15s 7ms/step - loss: 540866474.0405 - mean_squared_error: 540866474.0405 - val_loss: 4368593059840.0000 - val_mean_squared_error: 4368593059840.0000
Epoch 43/100
2210/2210 [==============================] - 15s 7ms/step - loss: 1053505187.1551 - mean_squared_error: 1053505187.1551 - val_loss: 4273901666304.0000 - val_mean_squared_error: 4273901666304.0000
Epoch 44/100
2210/2210 [==============================] - 12s 5ms/step - loss: 339979697.6255 - mean_squared_error: 339979697.6255 - val_loss: 4131650273280.0000 - val_mean_squared_er

Epoch 80/100
2210/2210 [==============================] - 13s 6ms/step - loss: 487164858.3903 - mean_squared_error: 487164858.3903 - val_loss: 3423620562944.0000 - val_mean_squared_error: 3423620562944.0000
Epoch 81/100
2210/2210 [==============================] - 12s 6ms/step - loss: 452014700.3833 - mean_squared_error: 452014700.3833 - val_loss: 3004542484480.0000 - val_mean_squared_error: 3004542484480.0000
Epoch 82/100
2210/2210 [==============================] - 13s 6ms/step - loss: 356733344.7304 - mean_squared_error: 356733344.7304 - val_loss: 3352312938496.0000 - val_mean_squared_error: 3352312938496.0000
Epoch 83/100
2210/2210 [==============================] - 12s 6ms/step - loss: 353982629.8674 - mean_squared_error: 353982629.8674 - val_loss: 3480919998464.0000 - val_mean_squared_error: 3480919998464.0000
Epoch 84/100
2210/2210 [==============================] - 11s 5ms/step - loss: 403486334.8746 - mean_squared_error: 403486334.8746 - val_loss: 3039833620480.0000 - val_mean

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

pred_train = model.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train)))

pred = model.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred)))